<a href="https://colab.research.google.com/github/danielvijill/geothermal/blob/main/geosite_temperature_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de predicción para temperaturas de sitios geotérmicos

En este modelo sencillo se utiliza una red neuronal completamente conectada con dos capas ocultas, entrenada para predecir una única salida continua a partir de dos entradas. Este modelo se utiliza para predecir las temperaturas de manifestaciones termales a partir de su localización en coordenadas geográficas. La base de datos que se carga contiene 209 instancias de las cuales se conoce la temperatura de 105 de ellas. La tarea del modelo es la de predecir el valor de las temperaturas faltantes.

### Importando librerías y cargando la base de datos

In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
folder_path = '/content/drive/MyDrive/Colab Notebooks'

In [42]:
csv_file_path = f'{folder_path}/sites.csv'
df = pd.read_csv(csv_file_path)
print(df.head())

   ID  Nombre                Cuadrante Departamento Coordenadas UTM Grid  \
0   1     NaN                ARAMECINA    F MORAZAN                44/11   
1   2     NaN                ARAMECINA    F MORAZAN                45/14   
2   3     NaN                   ARENAL         YORO              100/051   
3   4     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                99/00   
4   5     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                00/99   

   Latitud  Longitud  Temperatura Min  Temperatura Max  
0   13.670   -87.509              NaN              NaN  
1   13.707   -87.507              NaN              NaN  
2   15.422   -86.883             61.0             67.0  
3   14.546   -85.992              NaN              NaN  
4   14.505   -85.971              NaN              NaN  


### Generando el modelo

Filtramos el DataFrame en dos conjuntos: uno con sitios con temperaturas conocidas y uno con sitios sin temperaturas conocidas

In [43]:
# Filter the DataFrame into two sets: one with known temperatures and one with unknown temperatures
df_known = df.dropna(subset=['Temperatura Max'])
df_known = df_known.dropna(subset=['Latitud'])
df_known = df_known.dropna(subset=['Longitud'])
df_unknown = df[df['Temperatura Max'].isna()]

In [44]:
print(len(df))
print(len(df_known))

209
105


Definimos las características (entradas) y el objetivo (salida) para los datos conocidos

In [45]:
X_train = df_known[['Latitud', 'Longitud']].values
y_train = df_known['Temperatura Max'].values

Definir las características (entrada) para los datos desconocidos (prediremos el objetivo para estos)

In [46]:
X_predict = df_unknown[['Latitud', 'Longitud']].values

Creando un modelo simple de TensorFlow (una red neuronal de propagación hacia adelante), la capa definida como ``tf.keras.layers.Dense(1)`` es la capa de salida para la precicción de temperatura

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compilamos y entrenamos el modelo, utilizando Adam como optimizador y error medio cuadrático como función de pérdida

In [48]:
model.compile(optimizer='adam', loss='mse') # Using Mean Squared Error as loss

model.fit(X_train, y_train, epochs=100, verbose=0)

### Obteniendo datos

Ahora predecimos las temperaturas para los datos desconocidos con el modelo

In [49]:
predicted_temperatures = model.predict(X_predict)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


Actualizamos los datos en el DataFrame con los valores predichos por el modelo

In [50]:
df_predicted = df.copy()
df_predicted.loc[df_unknown.index, 'Temperatura Max'] = predicted_temperatures.flatten()
print(df_predicted.head())

   ID  Nombre                Cuadrante Departamento Coordenadas UTM Grid  \
0   1     NaN                ARAMECINA    F MORAZAN                44/11   
1   2     NaN                ARAMECINA    F MORAZAN                45/14   
2   3     NaN                   ARENAL         YORO              100/051   
3   4     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                99/00   
4   5     NaN  AZACUALPA-RIO GUAYAMBRE      OLANCHO                00/99   

   Latitud  Longitud  Temperatura Min  Temperatura Max  
0   13.670   -87.509              NaN        56.714199  
1   13.707   -87.507              NaN        56.721058  
2   15.422   -86.883             61.0        67.000000  
3   14.546   -85.992              NaN        55.978130  
4   14.505   -85.971              NaN        55.956345  


Finalmente guardamos el Data Frame como un archivo csv para su futura utilización

In [51]:
df_predicted.to_csv('predicted_temperatures.csv', index=False)